In [1]:
import os
import cv2
import threading
from mtcnn import MTCNN
from deepface import DeepFace

In [7]:
test_img = cv2.imread('known_faces/Deciano/Nathan.jpg')

test_result = DeepFace.extract_faces(test_img.copy())

print(test_result[0]['facial_area'])

{'x': 545, 'y': 191, 'w': 339, 'h': 339, 'left_eye': (774, 328), 'right_eye': (656, 325)}


In [8]:
for face in test_result:
    facial_area = face['facial_area']
    x = facial_area["x"]
    y = facial_area["y"]
    w = facial_area["w"]
    h = facial_area["h"]

    cur_face = test_img[y:y+h, x:x+w]
    cv2.imshow('current face', cur_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Draw the rectangle on the image
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow('Detected', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# using MTCNN through deepface
it is slow bud

In [9]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    facesCurFrame = DeepFace.extract_faces(img.copy(), detector_backend="mtcnn", enforce_detection=False)
    
    for faceLoc in facesCurFrame:
        facial_area = faceLoc.get("facial_area", {})
        x = facial_area.get("x", 0)
        y = facial_area.get("y", 0)
        w = facial_area.get("w", 0)
        h = facial_area.get("h", 0)

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Using MTCNN directly
wayyyy faster

In [10]:
detector = MTCNN(device="CPU:0")

result = detector.detect_faces(test_img)

print(result[0]['box'][0])

576


In [11]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    facesCurFrame = detector.detect_faces(img)
    
    for faceLoc in facesCurFrame:
        cv2.rectangle(img, (faceLoc['box'][0], faceLoc['box'][1]), (faceLoc['box'][0] + faceLoc['box'][2], faceLoc['box'][1] + faceLoc['box'][3]), (0, 255, 0), 2)

    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# MTCNN along with recognizer
I will be using facenet. However, in this file, I only used it to generate embeddings fro the known faces, and we are going to turn it into a pickle file

In [12]:
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot
from keras.models import load_model
from keras_facenet import FaceNet

import pickle
import cv2

In [13]:
# re declare the detectors
detector = MTCNN(device="CPU:0")

result = detector.detect_faces(test_img)

print(result[0]['box'][0])

576


In [14]:
# what if it detects a no-face image

no_face_image = cv2.imread('unknown.jpg')

no_face_result = detector.detect_faces(no_face_image)

print(len(no_face_result))

0


In [15]:
# what if it detects a face image

no_face_image = cv2.imread('known_faces/Komang/Bapallua.jpg')

no_face_result = detector.detect_faces(no_face_image)

print(len(no_face_result))
print(no_face_result)

1
[{'box': [451, 351, 177, 221], 'confidence': np.float64(0.9987026453018188), 'keypoints': {'nose': [np.int64(553), np.int64(490)], 'mouth_right': [np.int64(589), np.int64(526)], 'right_eye': [np.int64(588), np.int64(432)], 'left_eye': [np.int64(506), np.int64(443)], 'mouth_left': [np.int64(527), np.int64(534)]}}]


In [16]:
MyFaceNet = FaceNet()

In [21]:
folder = 'known_faces/'

for filename in os.listdir(folder):
    print(filename)
    for pic in os.listdir(os.path.join(folder, filename)):
        print(pic)

Deciano
Nathan.jpg
S__134266924_0.jpg
Devan
Devan.jpg
Putra.jpg
Komang
Bapallua.jpg
Komang.jpg
Rizal
Rizzal.jpg


In [25]:
folder = 'known_faces/'
database = {}

for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    database[filename] = []
    for pic in os.listdir(path):
        curimg_path = os.path.join(path, pic)
        curimg = cv2.imread(curimg_path)

        det_res = detector.detect_faces(curimg)

        if len(det_res) > 0:
            x, y, w, h = det_res[0]['box'][0], det_res[0]['box'][1], det_res[0]['box'][2],det_res[0]['box'][3]
        else:
            continue

        # cv2.rectangle(curimg, (x, y), (x+w, y+h), (0,255,0), 2)
        # cv2.imshow(filename, curimg)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows

        img_input = cv2.cvtColor(curimg, cv2.COLOR_BGR2RGB)
        img_input = Image.fromarray(img_input)
        img_array = np.asarray(img_input)

        face = img_array[y:y+h, x:x+w]
        face = Image.fromarray(face)
        face = face.resize((160,160))
        face = np.asarray(face)

        face = np.expand_dims(face, axis = 0)
        signature = MyFaceNet.embeddings(face)

        database[filename].append(signature)

database


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


{'Deciano': [array([[ 6.60134703e-02, -5.94949257e-03,  1.96792148e-02,
           2.59436201e-02,  5.63666644e-03, -1.26718869e-02,
          -4.36433293e-02,  3.93355675e-02, -2.61749998e-02,
          -2.13278830e-02,  1.32804280e-02,  5.25996648e-02,
           4.16722223e-02,  5.49176559e-02, -1.91174950e-02,
          -1.00719128e-02, -8.85193609e-03, -2.67344806e-02,
          -6.15554936e-02,  2.48820316e-02,  3.34780142e-02,
          -6.05384866e-03, -1.77759267e-02, -6.17305562e-02,
           1.58536602e-02,  1.76797081e-02,  3.79681960e-02,
          -3.04797012e-02,  6.08557276e-02, -2.82250717e-02,
           9.25172418e-02,  4.13030423e-02,  4.82909121e-02,
           2.73779146e-02, -2.26087309e-02, -6.94537461e-02,
          -1.28251076e-01, -5.57775646e-02, -6.70960397e-02,
           2.54648775e-02, -8.02996233e-02, -3.82065177e-02,
           5.95780136e-03, -1.53409503e-02, -2.64620502e-02,
           1.44574419e-02,  2.51876265e-02,  8.98099132e-03,
          -7.

In [26]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [34]:
# for name in database:
#     print(name)
#     for emb in database[name]:
#         print(emb)

print(len(database.items()))

4
